# MultiBand Single Pass (MBSP) Demonstration

This notebook demonstrates a basic implementation of the MBSP algorithm
described in *Varon et al. (2021)* for detecting large methane plumes using
Sentinel‑-2 imagery. The MBSP method compares band‑11 and band‑12
reflectances for a single scene to retrieve methane column enhancements.
We keep core routines in `src/mbsp.py` for reuse, while image selection
and experimentation remain in the notebook.

## MBSP versus MBMP

MBSP uses two spectral bands from a single acquisition. A scaling coefficient
is fitted between bands 11 and 12 over the scene and the fractional absorption
is computed as:

\begin{align}
R_{MBSP} = \frac{c R_{12} - R_{11}}{R_{11}}
\end{align}

where $c$ is the slope from fitting $R_{12}$ to $R_{11}$. This approach relies
on the surface behaving similarly in both bands.

The MultiBand MultiPass (MBMP) technique performs the MBSP retrieval on two
different dates and subtracts them to reduce artifacts. MBMP generally provides
better precision when a good plume‑free reference image is available.

In [ ]:
import datetime as dt

import ee
import geemap

# Authenticate with Earth Engine
ee.Authenticate()
ee.Initialize()


## Helper Functions

In [ ]:
def mask_s2_clouds(image: ee.Image) -> ee.Image:
    """Mask clouds using the QA60 band."""
    qa = image.select("QA60")
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    masked = image.updateMask(mask).divide(10000)
    return masked.copyProperties(image, image.propertyNames())  # <-- keep metadata!

In [ ]:
def mbsp_fractional_image(image: ee.Image, region: ee.Geometry) -> ee.Image:
    """
    Compute the MBSP (Multi-Band Single-Pass) fractional methane signal
    for a Sentinel-2 scene following Varon et al. 2021 (Atmos. Meas. Tech.).

    MBSP definition
    ----------------
        R_MBSP = ( c · R12  –  R11 ) / R11

    where
        R11, R12 : TOA reflectances for Sentinel-2 bands 11 (1610 nm) and 12 (2190 nm)
        c        : scene-wide slope from a zero-intercept linear regression of
                   R11 on R12 (c = Σ R11·R12 / Σ R12²)

    R_MBSP isolates the methane-induced absorption in band 12 by:
      1) Bringing band 12 onto the radiometric scale of band 11 (multiplying by c)
      2) Differencing and normalising by R11.

    A *negative* R_MBSP indicates that band 12 is darker than expected and is
    therefore consistent with CH₄ absorption.

    Parameters
    ----------
    image  : ee.Image
        Sentinel-2 L1C/L2A image containing bands ‘B11’ and ‘B12’.
    region : ee.Geometry
        Area (ideally plume-free) used to derive the scene-wide slope *c*.

    Returns
    -------
    ee.Image
        Single-band image named ‘R’ holding the pixel-wise MBSP signal.
        The fitted slope *c* is attached as image property ‘slope’.
    """
    # --- 1. Numerator and denominator for the regression slope -----------------
    # c = Σ( R11 * R12 ) / Σ( R12² )
    num_img = image.select("B11").multiply(image.select("B12"))  # R11·R12
    den_img = image.select("B12").multiply(image.select("B12"))  # R12²

    num_sum = num_img.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=region, scale=20, bestEffort=True
    )
    den_sum = den_img.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=region, scale=20, bestEffort=True
    )

    # Convert to ee.Number for further arithmetic
    slope = ee.Number(num_sum.get("B11")).divide(ee.Number(den_sum.get("B12")))

    # --- 2. Per-pixel MBSP field ----------------------------------------------
    mbsp = (
        image.select("B12")
        .multiply(slope)  # c · R12
        .subtract(image.select("B11"))
        .divide(image.select("B11"))
        .rename("R")
    )

    # Embed the slope for traceability
    return mbsp.set({"slope": slope})


## Image Selection

In [ ]:
# Location and date range of interest
lat, lon = 31.6585, 5.9053  # Hassi Messaoud example
start = dt.date(2019, 10, 1)
end = dt.date(2019, 10, 15)

point = ee.Geometry.Point(lon, lat)
collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterDate(str(start), str(end))
    .filterBounds(point)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .sort("system:time_start")
    .map(mask_s2_clouds)
)

images = collection.toList(collection.size())
count = images.size().getInfo()
print(f"Found {count} images")

In [ ]:
if count:
    region = point.buffer(1000).bounds()
    m = geemap.Map(center=(lat, lon), zoom=12)
    for i in range(count):
        img = ee.Image(images.get(i))
        date = ee.Date(img.get("system:time_start")).format("YYYY-MM-dd").getInfo()
        r_img = mbsp_fractional_image(img, region)
        m.addLayer(
            r_img,
            {"min": -0.05, "max": 0.05, "palette": ["blue", "white", "red"]},
            f"{date} fractional",
            True,
        )
        rgb = img.select(["B4", "B3", "B2"])
        m.addLayer(rgb, {"min": 0, "max": 0.3}, f"{date} RGB", False)

    styled_pt = ee.FeatureCollection([point]).style(
        **{
            "color": "green",  # outline & fill
            "fillColor": None,  # no fill
            "pointSize": 8,  # pixel radius of the dot
        }
    )
    m.addLayer(styled_pt, {}, "centre pt", True)

In [ ]:
m

The map above contains fractional signal layers for each scene interleaved
with true-color imagery. Fractional layers are visible by default to help
identify methane plumes over time.

In [ ]:
import ee
import xarray as xr
import numpy as np


def k_lookup(lut_ds, band, gas, sza, vza, pres):
    """Return scalar k_{j,g} from local xarray LUT."""
    return float(
        lut_ds.k_prime.sel(band=band, gas=gas)
        .interp(sza=sza, vza=vza, pres=pres)
        .values
    )


def safe_sum(img, band, region, scale):
    """Return ee.Number(sum) or raise a readable error."""
    value = (
        img.select(band)
        .reduceRegion(
            reducer=ee.Reducer.sum(), geometry=region, scale=scale, bestEffort=True
        )
        .values()
        .get(0)
    )
    return ee.Number(value)


# ------------------------------------------------------------------
# 1.  Load k-LUTs (done once per session on the client side)
# ------------------------------------------------------------------
lutA = xr.open_dataset("../data/lookup/k_S2A_v1.nc")  # for S2A images
lutB = xr.open_dataset("../data/lookup/k_S2B_v1.nc")  # for S2B images


def _interp_k(lut, sza, vza, pres, band, gas):
    """Trilinear interpolation helper – returns scalar k_{j,g}."""
    return (
        lut.k_prime.sel(band=band, gas=gas).interp(sza=sza, vza=vza, pres=pres).item()
    )


# ------------------------------------------------------------------
# 2.  Cloud mask helpers
# ------------------------------------------------------------------
def _qa60_mask(img):
    """
    Returns a Boolean cloud mask using the QA60 bits.
    Handles the cases where QA60 is missing or not integer.
    """
    has_QA60 = img.bandNames().contains("QA60")

    # If QA60 is absent, return an all-false mask (no pixels masked)
    def _no_qa():
        return ee.Image(0).rename("nomask").eq(1)  # always False

    # Normal path
    def _use_qa():
        qa = (
            img.select("QA60")
            .unmask(0)  # fill nulls with 0
            .toUint16()  # <-- CAST to integer
        )
        opaque = qa.bitwiseAnd(1 << 10).neq(0)
        cirrus = qa.bitwiseAnd(1 << 11).neq(0)
        return opaque.Or(cirrus)

    return ee.Algorithms.If(has_QA60, _use_qa(), _no_qa())


def _spectral_cloud_mask(img):
    # Very bright in blue or cirrus band  → cloud
    blue = img.select("B1").gt(0.15)  # 60 m aerosol band
    cir = img.select("B10").gt(0.02)  # high reflectance at 1375 nm
    green = img.select("B3").gt(0.15)
    return blue.Or(green).Or(cir)


def _mask_clouds(img):
    # CAST -- make sure the output of _qa60_mask is an ee.Image
    qa_mask = ee.Image(_qa60_mask(img))  # ★ this was the culprit
    mask = qa_mask.Or(_spectral_cloud_mask(img))  # now both operands are Images
    return img.updateMask(mask.Not())


# ------------------------------------------------------------------
# 3.  Main CH₄ retrieval
# ------------------------------------------------------------------
def ch4_column_mbsp_corrected(
    image: ee.Image, region: ee.Geometry, surface_band: str = "B8A"
) -> ee.Image:
    # ---------- Client-side metadata fetch -------------------------
    sat_name = image.get("SPACECRAFT_NAME").getInfo()  # 'Sentinel-2A' / 'Sentinel-2B'
    sza_val = image.get("MEAN_SOLAR_ZENITH_ANGLE").getInfo()  # degrees
    vza_val = 0.0  # assume nadir
    pres_val = 1013.0  # hPa (sea-level)

    lut_ds = lutA if "2A" in sat_name else lutB

    # k-coefficients (scalars) -------------------------------------
    k9_H2O = ee.Number(k_lookup(lut_ds, "B9", "H2O", sza_val, vza_val, pres_val))
    k11_H2O = ee.Number(k_lookup(lut_ds, "B11", "H2O", sza_val, vza_val, pres_val))
    k12_H2O = ee.Number(k_lookup(lut_ds, "B12", "H2O", sza_val, vza_val, pres_val))
    k11_CO2 = ee.Number(k_lookup(lut_ds, "B11", "CO2", sza_val, vza_val, pres_val))
    k12_CO2 = ee.Number(k_lookup(lut_ds, "B12", "CO2", sza_val, vza_val, pres_val))
    k11_CH4 = ee.Number(k_lookup(lut_ds, "B11", "CH4", sza_val, vza_val, pres_val))
    k12_CH4 = ee.Number(k_lookup(lut_ds, "B12", "CH4", sza_val, vza_val, pres_val))

    # ---------- 3.2  Mask clouds -----------------------------------
    img = _mask_clouds(image)

    # ---------- 3.3  Water-vapour solve -----------------------------
    # Continuum-scale B9 with B8A inside region (per-scene slope)
    cw_num = safe_sum(img.select("B9"), 0, region, 60)
    cw_denom = safe_sum(img.select(surface_band), 0, region, 20)
    c_w = cw_num.divide(cw_denom)

    # ΔR9 (expected minus observed) – note resolution difference (resample)
    R9_exp = img.select(surface_band).multiply(c_w).resample("bicubic")
    dR9 = R9_exp.subtract(img.select("B9"))
    dW = dR9.divide(k9_H2O)  # mol m⁻²

    # Apply water correction
    R11_w = img.select("B11").add(dW.multiply(k11_H2O))
    R12_w = img.select("B12").add(dW.multiply(k12_H2O))

    # ---------- 3.4  CO₂ solve & correction -------------------------
    # Continuum estimate for the (already water-corrected) B11
    sum_R11w = safe_sum(R11_w, 0, region, 20)  # already Number
    sum_B8A = safe_sum(img.select(surface_band), 0, region, 20)

    c_c = sum_R11w.divide(sum_B8A.max(1e-6))  # no AttributeError
    R11_cont = img.select(surface_band).multiply(c_c)
    dR11 = R11_cont.subtract(R11_w)
    dC = dR11.divide(k11_CO2)

    # Apply CO₂ correction to band 12
    R11_wc = R11_w  # fully corrected B11
    R12_wc = R12_w.add(dC.multiply(k12_CO2))

    # ---------- 3.5  MBSP methane residual --------------------------
    # Scene-wide slope c over the (water+CO₂ corrected) bands
    num = (
        R11_wc.multiply(R12_wc)
        .reduceRegion(ee.Reducer.sum(), region, 20, bestEffort=True)
        .values()
        .get(0)
    )
    den = (
        R12_wc.multiply(R12_wc)
        .reduceRegion(ee.Reducer.sum(), region, 20, bestEffort=True)
        .values()
        .get(0)
    )
    c_mbsp = ee.Number(num).divide(ee.Number(den))

    R_mbsp = R12_wc.multiply(c_mbsp).subtract(R11_wc).divide(R11_wc).rename("R")

    # ---------- 3.6  Column anomaly (mol m-²) -----------------------
    denom = k12_CH4.subtract(c_mbsp.multiply(k11_CH4))
    dCH4 = R_mbsp.multiply(-1).divide(denom).rename("dCH4")

    # ---------- 3.7  Package & return -------------------------------
    return ee.Image.cat([R_mbsp, dCH4]).set(
        {
            "slope": c_mbsp,
            "k11_H2O": k11_H2O,
            "k12_H2O": k12_H2O,
            "k11_CO2": k11_CO2,
            "k12_CO2": k12_CO2,
            "k11_CH4": k11_CH4,
            "k12_CH4": k12_CH4,
        }
    )


In [ ]:
if count:
    region = point.buffer(1000).bounds()
    m = geemap.Map(center=(lat, lon), zoom=12)
    for i in range(count):
        img = ee.Image(images.get(i))
        date = ee.Date(img.get("system:time_start")).format("YYYY-MM-dd").getInfo()
        # R is sensor-native and easy to sanity-check (no wind, no LUT assumptions).
        # dCH4 is what you need for physics (integrated-mass, kg h⁻¹ source rates) but depends on the absorption-coefficient lookup and geometry metadata; if those inputs are off, R lets you spot the issue quickly.
        r_img = ch4_column_mbsp_corrected(img, region).select("R")
        m.addLayer(
            r_img,
            {"min": -0.05, "max": 0.05, "palette": ["blue", "white", "red"]},
            f"{date} fractional",
            True,
        )
        rgb = img.select(["B4", "B3", "B2"])
        m.addLayer(rgb, {"min": 0, "max": 0.3}, f"{date} RGB", False)

    styled_pt = ee.FeatureCollection([point]).style(
        **{
            "color": "green",  # outline & fill
            "fillColor": None,  # no fill
            "pointSize": 8,  # pixel radius of the dot
        }
    )
    m.addLayer(styled_pt, {}, "centre pt", True)

In [ ]:
m

In [ ]:
import numpy as np
import xarray as xr
from scipy.ndimage import zoom
from offshore_methane.mbsp import invert_mbsp, S2_CONSTANTS

# lookup tables for absorption coefficients
lutA = xr.open_dataset('../data/lookup/k_S2A_v1.nc')
lutB = xr.open_dataset('../data/lookup/k_S2B_v1.nc')

def k_value(sensor, band, gas, sza, vza=0, pres=1013):
    lut = lutA if sensor == 'Sentinel-2A' else lutB
    return (
        lut.k_prime.sel(band=band, gas=gas)
        .interp(sza=sza, vza=vza, pres=pres)
        .item()
    )

def robust_scale(b11, b12, max_iter=3, thresh=3.0):
    mask = np.isfinite(b11) & np.isfinite(b12)
    c = np.sum(b11[mask] * b12[mask]) / np.sum(b12[mask] ** 2)
    for _ in range(max_iter):
        r = (c * b12 - b11) / b11
        sigma = np.nanstd(r[mask])
        new_mask = mask & (np.abs(r) < thresh * sigma)
        if np.array_equal(new_mask, mask):
            break
        mask = new_mask
        c = np.sum(b11[mask] * b12[mask]) / np.sum(b12[mask] ** 2)
    return c

def mbsp_single_scene(R8A, R9, R11, R12, sensor, sza, vza=0, pres=1013):
    # 1. resample band 9 to match 20 m bands
    if R9.shape != R8A.shape:
        zoom_factor = R8A.shape[0] / R9.shape[0]
        R9 = zoom(R9, zoom_factor, order=3)

    # 2. water vapour from B9/B8A ratio
    k9 = k_value(sensor, 'B9', 'H2O', sza, vza, pres)
    delta_tau = -np.log(np.clip(R9 / R8A, 1e-6, None))
    W = delta_tau / k9

    # 3. humidity-correct bands
    k11 = k_value(sensor, 'B11', 'H2O', sza, vza, pres)
    k12 = k_value(sensor, 'B12', 'H2O', sza, vza, pres)
    tau11 = k11 * W
    tau12 = k12 * W
    R11_dry = R11 * np.exp(tau11 - tau11.mean())
    R12_dry = R12 * np.exp(tau12 - tau12.mean())

    # 4. robust MBSP slope
    c = robust_scale(R11_dry, R12_dry)

    # 5. fractional signal
    rfrac = (c * R12_dry - R11_dry) / R11_dry

    # 6. methane column enhancement
    const = S2_CONSTANTS['A' if '2A' in sensor else 'B']
    delta = invert_mbsp(rfrac, const)
    return delta, c


In [ ]:
# Demonstration with synthetic arrays
rng = np.random.default_rng(0)
R8A = 0.3 + 0.1 * rng.random((20, 20))
R9 = 0.25 + 0.05 * rng.random((10, 10))
R11 = 0.32 + 0.05 * rng.random((20, 20))
R12 = 0.30 + 0.05 * rng.random((20, 20))
delta, c = mbsp_single_scene(R8A, R9, R11, R12, 'Sentinel-2A', sza=30)
print('scale factor:', round(float(c), 3))
print('mean column enhancement:', round(float(delta.mean()), 4), 'mol m-2')


In [ ]:
delta[:3, :3]